notes:  

annotations originally started with zooniverse platform. however, because of data quality issues from numerous annotators, we resorted to using only 3 annotators that were better trained and thus more consistent.  
there are three sets. one used for training the annotators, one used to assess agreement between annotators, and the third is the final set that annotators were able to annotate on their own assuming a certain level of agreement was reached.  
the training set consists of 30 randomly selected posts  
the agreement set consists of 100 randomly selected posts. this set also happens to be the same set that is being annotated by domain experts.  
the final sets consist of 125 posts each. this would give us approximately 500 total posts.   


because of the issues with the platform post ids. i need to check to see if there is any overlap in the training and agreement sets using the text itself. 

In [4]:
import pandas as pd
import numpy as np
import os

# data were randomly selected from posts that were already annotated with zooniverse but had disagreement initially.  
training_dat = pd.read_csv("../data/annotation_sets/annotation_training.csv") 
# rename aggreed to subject
training_dat = training_dat.rename(columns={"aggreed":"subject"})
# keep only columns we need: 5:20
training_dat = training_dat[training_dat.columns[4:20]]
expert_dat = pd.read_csv("../data/annotation_sets/test_set.csv")

In [5]:
import pickle

subject_dat = pickle.load(open("../data/zooni/subject_dat.pkl", "rb"))

In [6]:
expert_dat['overlap'] = ''
for post in expert_dat['text']:
    if post in training_dat['text'].values:
        expert_dat['overlap'][expert_dat['text'] == post] = 'yes'
    else:
        expert_dat['overlap'][expert_dat['text'] == post] = 'no'

In [7]:
print(expert_dat['overlap'].value_counts())

no     85
yes    15
Name: overlap, dtype: int64


In [12]:
# assign ids
expert_dat['id'] = pd.merge(expert_dat, subject_dat, on='text')['subject_id']
training_dat['id'] = pd.merge(training_dat, subject_dat, on='text')['subject_id']

In [15]:
# add cols
expert_dat['connection'] = ''
expert_dat['subject'] = ''
objectives = training_dat.columns[3:16]
# add cols
for objective in objectives:
    expert_dat[objective] = ''
# export expert dat
expert_dat.to_csv("../data/expert_dat.csv", index=False)

---

## Datasets for each annotator

In [8]:
posts = pd.read_json("../data/raw_dat/all_drug_posts.json")

In [16]:
# select random 500 posts - not [removed] and not empty
seed = 2024
posts_sample = posts[(posts['text'] != '[removed]') & (posts['text'] != '')].sample(400, random_state=seed)

In [17]:
print(posts['subreddit'].value_counts()/len(posts))

cocaine            0.355252
opiates            0.273208
benzodiazepines    0.241414
stims              0.130126
Name: subreddit, dtype: float64


In [18]:
# get proportion of each subreddit in sample
print(posts_sample['subreddit'].value_counts()/len(posts_sample))

opiates            0.320
cocaine            0.285
benzodiazepines    0.250
stims              0.145
Name: subreddit, dtype: float64


In [19]:
posts_sample['all_text'] = posts_sample['title'] + ' ' + posts_sample['text']

In [20]:
# are any of the test set in the sample?
overlap = []
for post in expert_dat['text']:
    if post in posts_sample['all_text'].values:
        overlap.append('yes')
    else:
        overlap.append('no')

print(pd.Series(overlap).value_counts())

no    100
dtype: int64


In [21]:
# export sample
posts_sample[['subreddit', 'all_text']].to_csv("../data/annotation_sets/posts_training.csv", index=False)

In [22]:
# save training set
import pickle
pickle.dump(posts_sample, open("../data/posts_training.pkl", "wb"))